---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    with open("university_towns.txt") as q:
        df=q.readlines()
    df=[a.rstrip() for a in df]
    list3=[]
    for i in df:
        if i[-6:] == '[edit]':
            list1=i[:-6]
        elif '(' in i:
            list2=i[:i.index('(')-1]
            list3.append([list1,list2])
        else:
            list2=i
            list3.append([list1,list2])
    df1 = pd.DataFrame(list3, columns=['State','RegionName'])
        
   
    
    return df1
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df=pd.read_excel('gdplev.xls',skiprows=200)
    df=df.rename(columns={'1995q1':'quater',7545.3:'GDP'})
    df=df[['quater','GDP']]
    for i in range(0,df.shape[0]-1):
        if(df.iloc[i-2][1]>df.iloc[i-1][1])and(df.iloc[i-1][1]>df.iloc[i][1]):
            a=df.iloc[i-3][0]
    
    
    return a
get_recession_start()

'2008q3'

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df=pd.read_excel('gdplev.xls',skiprows=200)
    df=df.rename(columns={'1995q1':'quater',7545.3:'GDP'})
    df=df[['quater','GDP']]
    for i in range(4,len(df)):
        if (df.iloc[i-4][1]>df.iloc[i-3][1]) and (df.iloc[i-3][1]>df.iloc[i-2][1]) and (df.iloc[i-2][1]<df.iloc[i-1][1]) and (df.iloc[i-1][1]<df.iloc[i][1]):
            a=df.iloc[i][0]
       
    return a
get_recession_end()

'2009q4'

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df=pd.read_excel('gdplev.xls',skiprows=200)
    df=df.rename(columns={'1995q1':'quater',7545.3:'GDP'})
    df=df[['quater','GDP']]
    for i in range(4,len(df)):
        if (df.iloc[i-4][1]>df.iloc[i-3][1]) and (df.iloc[i-3][1]>df.iloc[i-2][1]) and (df.iloc[i-2][1]<df.iloc[i-1][1]) and (df.iloc[i-1][1]<df.iloc[i][1]):
            a=df.iloc[i-2][0]
       
    return a
get_recession_bottom()
    


'2009q2'

In [26]:
#df=pd.read_csv('City_Zhvi_AllHomes.csv')
#df['State'].replace('states',inplace=True)
#df= df.set_index(["State","RegionName"])
#df=df.loc[:,"2000-01":"2016-08"]
#if col.endswith(("01","02","03")):
       #     a=col[:4]+"q1"
      #  elif col.endswith(("04","05","06")):
     #       a=col[:4]+"q2"
    #    elif col.endswith(("07","08","09")):
   #         a=col[:4]+"q3"
  #      else:
 #           a=col[:4]+"q4"
#df


In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    month_to_qtr = {'01':'q1','02':'q1','03':'q1','04':'q2','05':'q2','06':'q2',
                    '07':'q3','08':'q3','09':'q3','10':'q4','11':'q4','12':'q4'}
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    df['State'] = df['State'].replace(states)
    df = (df.drop(['Metro','CountyName','SizeRank'], axis = 1)
            .set_index(['RegionID','State','RegionName'])
            .stack()
            .reset_index()
            .rename(columns={'level_3':'Year-Mo',
                                          0:'MedianPrice'}))
    df['YearQtr'] = df['Year-Mo'].str[0:4] + df['Year-Mo'].str[5:].map(month_to_qtr)
        
    df = df[df['YearQtr'] >= '2000q1']
    df = (df.drop(['Year-Mo'], axis = 1)
             .groupby(['RegionID','State','RegionName','YearQtr'])
             .mean()
             .unstack())
    df.index = df.index.droplevel()
    df.columns = df.columns.droplevel()    
    del df.columns.name
    return df
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New Jersey,Absecon,113500.000000,117633.333333,118033.333333,118833.333333,121033.333333,121700.000000,123466.666667,125766.666667,128433.333333,134933.333333,...,174500.000000,171333.333333,168133.333333,167033.333333,166366.666667,164033.333333,158200.000000,1.543000e+05,1.518667e+05,148750.0
Maryland,Accokeek,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,311066.666667,323766.666667,335000.000000,333133.333333,331700.000000,328266.666667,321866.666667,3.190667e+05,3.305000e+05,331400.0
Oklahoma,Ada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,71233.333333,72333.333333,75100.000000,75466.666667,78966.666667,82833.333333,85700.000000,8.720000e+04,9.263333e+04,97200.0
Georgia,Adairsville,85966.666667,87300.000000,85100.000000,86033.333333,86466.666667,86200.000000,88666.666667,88733.333333,92166.666667,94766.666667,...,100066.666667,100033.333333,101133.333333,99933.333333,102133.333333,106500.000000,111500.000000,1.148667e+05,1.208000e+05,124400.0
Tennessee,Adams,109833.333333,111100.000000,112133.333333,113100.000000,114500.000000,115966.666667,115433.333333,115566.666667,117500.000000,118733.333333,...,188533.333333,193566.666667,192200.000000,191400.000000,193133.333333,192800.000000,197533.333333,2.002333e+05,1.994333e+05,200900.0
Illinois,Addison,164600.000000,168366.666667,172433.333333,176133.333333,179500.000000,184100.000000,189233.333333,192766.666667,196200.000000,199900.000000,...,198600.000000,201966.666667,203666.666667,206166.666667,208266.666667,206933.333333,207700.000000,2.096667e+05,2.111000e+05,216900.0
South Carolina,Aiken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,104866.666667,108266.666667,112966.666667,115300.000000,117366.666667,117366.666667,118333.333333,1.181667e+05,1.176333e+05,118950.0
Michigan,Albion,62500.000000,63533.333333,64966.666667,66533.333333,66933.333333,68566.666667,68600.000000,68900.000000,70366.666667,71366.666667,...,62766.666667,64866.666667,66200.000000,64466.666667,63933.333333,63366.666667,63266.666667,6.513333e+04,6.666667e+04,67350.0
California,Alta,145933.333333,151966.666667,153066.666667,157200.000000,166166.666667,171233.333333,180666.666667,189600.000000,193033.333333,197200.000000,...,235100.000000,247566.666667,257400.000000,279000.000000,272066.666667,274433.333333,275800.000000,2.812333e+05,2.915333e+05,296850.0


In [11]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    start = get_recession_start()
    bottom = get_recession_bottom()
    UTowns_df = get_list_of_university_towns()
    quart_df = convert_housing_data_to_quarters()
    
    quart_df['De-Gr'] = quart_df[start] - quart_df[bottom]
    
    UTowns_df['Check'] = 'Yes'
    quart_df = pd.merge(quart_df, UTowns_df, how = 'left', left_index=True, right_on=['State', 'RegionName'])
    quart_df['Check'] = quart_df['Check'].replace(np.nan, 'No')
    quart_df['De-Gr'] = quart_df['De-Gr'].fillna('...')
    quart_df = quart_df[quart_df['De-Gr'] != '...']
    Yes = quart_df[quart_df['Check'] == 'Yes']
    No = quart_df[quart_df['Check'] == 'No']
    ttest = ttest_ind(Yes['De-Gr'], No['De-Gr'])[1] 
    
    if ttest < 0.01:
        different = True
    else:
        different = False
    
    if Yes['De-Gr'].mean() < No['De-Gr'].mean():
        better = "university town"
    else:
        better = "non-university town"
    
    return (different , ttest, better)
run_ttest()

(True, 0.0043252148535114473, 'university town')